# TKU Report (BMC Discovery)

This notebook reads the metadata produced by DisMAL’s TKU log to show the most recent uploads.
It loads entries from `raw_exports/<appliance>/tku_log.txt` (generated by the CLI) and writes `tku.csv`
under the standard output folder for each appliance.


## Requirements

Only `PyYAML` is required to read configuration; parsing uses the standard library. Uncomment
the next cell if you need to install it.


In [ ]:
# %pip install -q pyyaml pandas

from pathlib import Path
import pandas as pd
import yaml


## Configuration (from config.yaml)

Locates the repository root, reads configuration, and determines the raw export/output
directories.


In [ ]:
def load_config_params(start: Path, appliance_name: str = None, appliance_index: int = 0) -> dict:
    def _find_repo_root(path: Path) -> Path:
        for candidate in [path] + list(path.parents):
            if (candidate / 'config.yaml').exists():
                return candidate
        return path.parent

    def _slugify(value: str) -> str:
        return ''.join(ch if ch.isalnum() else '_' for ch in value).strip('_').lower() or 'default'

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'

    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    appliances = cfg.get('appliances') or []
    selected = None
    if isinstance(appliances, list) and appliances:
        if appliance_name:
            selected = next((a for a in appliances if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = appliances[int(appliance_index)]
            except Exception:
                selected = appliances[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    export_name = ((selected or {}).get('name') or appliance_name or sanitized)
    raw_export_dir = repo_root / 'raw_exports' / _slugify(export_name)

    return {
        'repo_root': repo_root,
        'config_path': config_path,
        'cfg': cfg,
        'selected': selected,
        'target': target,
        'output_dir': output_dir,
        'raw_export_dir': raw_export_dir,
    }


## Initialise Instances


In [ ]:
twprod = load_config_params(Path.cwd(), appliance_name='prod')
print('Prod Target  :', twprod['target'])
print('Prod Exports :', twprod['raw_export_dir'])
print('Prod Output  :', twprod['output_dir'])

twdev = load_config_params(Path.cwd(), appliance_name='dev')
print('Dev Target   :', twdev['target'])
print('Dev Exports  :', twdev['raw_export_dir'])
print('Dev Output   :', twdev['output_dir'])


## Parse exports

`tku_log.txt` contains timestamped entries generated by the CLI (one per upload).
We parse the file and take the most recent TKU / EDP / Storage entry.


In [ ]:
LOG_FILENAME = 'tku_log.txt'
COMPONENTS = [('latest_tku', 'TKU'), ('latest_edp', 'EDP'), ('latest_storage', 'Storage')]

def parse_tku_log(path: Path) -> dict:
    info = {key: 'Not installed' for key, _ in COMPONENTS}
    if not path.exists():
        return info
    with open(path, 'r', encoding='utf-8', errors='replace') as fh:
        for line in fh:
            parts = line.strip().split(',', 2)
            if len(parts) < 3:
                continue
            _, key, value = parts
            key = key.strip().lower()
            value = value.strip() or 'Not installed'
            if key in info:
                info[key] = value
    return info

def build_tku_dataframe(instance: dict) -> pd.DataFrame:
    log_path = instance['raw_export_dir'] / LOG_FILENAME
    info = parse_tku_log(log_path)
    rows = [
        {
            'Discovery Instance': instance['target'],
            'Component': label,
            'Version': info.get(key, 'Not installed'),
        }
        for key, label in COMPONENTS
    ]
    return pd.DataFrame(rows)

prod_df = build_tku_dataframe(twprod)
print(twprod['target'])
display(prod_df)

# Attempt to load dev appliance; if configuration lacks it, reuse prod info
try:
    dev_df = build_tku_dataframe(twdev)
    print(twdev['target'])
    display(dev_df)
except Exception as exc:
    print('Dev export missing or misconfigured:', exc)
    dev_df = prod_df.copy()
    dev_df['Discovery Instance'] = '(dev missing)'


## Save CSV


In [ ]:
OUTPUT_FILE = 'tku.csv'

prod_path = twprod['output_dir'] / OUTPUT_FILE
prod_df.to_csv(prod_path, index=False)
print(f'Saved prod TKU info to {prod_path}')

if 'dev_df' in globals():
    dev_path = twdev['output_dir'] / OUTPUT_FILE
    dev_df.to_csv(dev_path, index=False)
    print(f'Saved dev TKU info to {dev_path}')


---
### Notes
- The CLI appends to `tku_log.txt` each time TKU/EDP/Storage content is downloaded.
- If the log is missing, this notebook reports all components as `Not installed`.
